# Pickling cuML Models for Persistence

This notebook demonstrates simple pickling of both single-GPU and multi-GPU cuML models for persistence

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Single GPU Model Pickling

All single-GPU estimators are pickleable. The following example demonstrates the creation of a synthetic dataset, training, and pickling of the resulting model for storage. Trained single-GPU models can also be used to distribute the inference on a Dask cluster, which the `Distributed Model Pickling` section below demonstrates.

In [2]:
from cuml.datasets import make_blobs

X, y = make_blobs(n_samples=50,
                  n_features=10,
                  centers=5,
                  cluster_std=0.4,
                  random_state=0)

In [3]:
from cuml.cluster import KMeans

model = KMeans(n_clusters=5)

model.fit(X)

KMeans()

In [4]:
import pickle

pickle.dump(model, open("kmeans_model.pkl", "wb"))

In [5]:
model = pickle.load(open("kmeans_model.pkl", "rb"))

In [6]:
model.cluster_centers_

array([[-5.7684636 ,  2.3276033 , -3.7457771 , -1.8541754 , -5.1695833 ,
         7.667088  ,  2.7118318 ,  8.495609  ,  1.7038484 ,  1.1884269 ],
       [ 4.647688  ,  8.37788   , -9.0705805 ,  9.459332  ,  8.450422  ,
        -1.0210547 ,  3.3920872 , -7.8629856 , -0.75276613,  0.48384127],
       [-2.9414437 ,  4.6401706 , -4.5027533 ,  2.2855108 ,  1.644645  ,
        -2.4937892 , -5.2241607 , -1.5499196 , -8.063638  ,  2.816936  ],
       [-4.2710767 ,  5.5611653 , -5.6640916 , -1.8229512 , -9.2925    ,
         0.73028314,  4.4586773 , -2.8876226 , -5.1257744 ,  9.694357  ],
       [ 5.583742  , -4.1515303 ,  4.369667  , -3.0020504 ,  3.6388965 ,
        -4.3419113 , -3.318711  ,  6.503671  , -6.865036  , -1.0266497 ]],
      dtype=float32)

## Distributed Model Pickling

The distributed estimator wrappers inside of the `cuml.dask` are not intended to be pickled directly. The Dask cuML estimators provide a function `get_combined_model()`, which returns the trained single-GPU model for pickling. The combined model can be used for inference on a single-GPU, and the `ParallelPostFit` wrapper from the [Dask-ML](https://ml.dask.org/meta-estimators.html) library can be used to perform distributed inference on a Dask cluster.

In [7]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:38149 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 251.80 GiB


In [8]:
from cuml.dask.datasets import make_blobs

n_workers = len(client.scheduler_info()["workers"].keys())

X, y = make_blobs(n_samples=5000, 
                  n_features=30,
                  centers=5, 
                  cluster_std=0.4, 
                  random_state=0,
                  n_parts=n_workers*5)

X = X.persist()
y = y.persist()

In [9]:
from cuml.dask.cluster import KMeans

dist_model = KMeans(n_clusters=5)

In [10]:
dist_model.fit(X)

In [11]:
import pickle

single_gpu_model = dist_model.get_combined_model()
pickle.dump(single_gpu_model, open("kmeans_model.pkl", "wb"))

In [12]:
single_gpu_model = pickle.load(open("kmeans_model.pkl", "rb"))

In [13]:
single_gpu_model.cluster_centers_

array([[ 4.809875  ,  8.422671  , -9.239023  ,  9.379142  ,  8.499883  ,
        -1.0592821 ,  3.3437853 , -7.8026133 , -0.5946334 ,  0.264476  ,
         5.5073953 , -4.106979  ,  4.2890787 , -2.817205  ,  3.6150157 ,
        -4.1612997 , -3.620963  ,  6.218531  , -6.9460464 , -1.0828305 ,
        -5.8267713 ,  2.2258773 , -3.8601215 , -1.6974076 , -5.313416  ,
         7.579577  ,  2.9187481 ,  8.540427  ,  1.5523201 ,  1.0841807 ],
       [-2.8941855 ,  4.474188  , -4.447566  ,  2.3820994 ,  1.7478832 ,
        -2.5046248 , -5.208329  , -1.6937679 , -8.134757  ,  2.6468298 ,
        -4.3163624 ,  5.5655394 , -5.7321997 , -1.7384953 , -9.34465   ,
         0.7084658 ,  4.435839  , -2.9009004 , -4.9486394 ,  9.695301  ,
         8.366521  , -6.2474546 , -6.349472  ,  1.9546975 ,  4.1576157 ,
        -9.167905  ,  4.6070676 ,  8.788581  ,  6.8644238 ,  2.2319865 ],
       [-4.6657143 , -9.558952  ,  6.6572275 ,  4.440131  ,  2.1730297 ,
         2.5904038 ,  0.58000994,  6.255038  , -8